In [0]:
from google.colab import files
files.upload()

In [0]:
!pip install -q kaggle

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

In [0]:
!kaggle datasets list

In [0]:
!kaggle datasets download -d chetankv/dogs-cats-images

In [0]:
!unzip dogs-cats-images.zip

In [0]:
!unzip "dog vs cat.zip"

In [0]:
from keras.applications.vgg16 import VGG16
from keras.layers import Input, Flatten, Dense
from keras.models import Model

def vgg2c():
	model = VGG16(weights='imagenet', include_top=False)

	#Create your own input format (here 3x200x200)
	input = Input(shape=(224,224,3),name = 'image_input')

	#Use the generated model 
	output_vgg16_conv = model(input)

	#Add the fully-connected layers 
	x = Flatten(name='flatten')(output_vgg16_conv)
	x = Dense(4096, activation='relu', name='fc1')(x)
	x = Dense(4096, activation='relu', name='fc2')(x)
	x = Dense(2, activation='softmax', name='predictions')(x)

	#Create your own model 
	my_model = Model(inputs=input, outputs=x)

	return my_model

In [0]:
dogs_train = []
cats_train = []
dogs_test = []
cats_test = []

dogs_train_target = []
cats_train_target = []
dogs_test_target = []
cats_test_target = []


for filename in glob.glob('dataset/training_set/dogs/*.jpg'): #assuming gif
  im = cv2.imread(filename)
  try:
    im = cv2.resize(im,(224,224))
    #im = np.expand_dims(im, axis=0)
    im = preprocess_input(im)
    dogs_train.append(im)
    dogs_train_target.append([1,0])
  except:
    pass

for filename in glob.glob('dataset/training_set/cats/*.jpg'): #assuming gif
  im = cv2.imread(filename)
  try:
    im = cv2.resize(im,(224,224))
    #im = np.expand_dims(im, axis=0)
    im = preprocess_input(im)
    cats_train.append(im)
    cats_train_target.append([0,1])
  except:
    pass

for filename in glob.glob('dataset/test_set/dogs/*.jpg'): #assuming gif
  im = cv2.imread(filename)
  try:
    im = cv2.resize(im,(224,224))
    #im = np.expand_dims(im, axis=0)
    im = preprocess_input(im)
    dogs_test.append(im)
    dogs_test_target.append([1,0])
  except:
    pass

for filename in glob.glob('dataset/test_set/cats/*.jpg'): #assuming gif
  im = cv2.imread(filename)
  try:
    im = cv2.resize(im,(224,224))
    #im = np.expand_dims(im, axis=0)
    im = preprocess_input(im)
    cats_test.append(im)
    cats_test_target.append([0,1])
  except:
    pass

dogs_train = np.array(dogs_train)
cats_train = np.array(cats_train)
dogs_test = np.array(dogs_test)
cats_test = np.array(cats_test)

dogs_train_target = np.array(dogs_train_target)
cats_train_target = np.array(cats_train_target)
dogs_test_target = np.array(dogs_test_target)
cats_test_target = np.array(cats_test_target)

train_set = np.concatenate((dogs_train, cats_train))
train_targets = np.concatenate((dogs_train_target, cats_train_target))

test_set = np.concatenate((dogs_test, cats_test))
test_targets = np.concatenate((dogs_test_target, cats_test_target))

train_set, train_targets = shuffle(train_set, train_targets)
test_set, test_targets = shuffle(test_set, test_targets)


In [0]:
import keras
import tensorflow as tf


config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 56} ) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)

In [0]:
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
import glob
import numpy as np
import cv2

model = vgg2c()

for layer in model.layers[:2]:
    layer.trainable=False

model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [0]:
model.fit(train_set, train_targets, batch_size = 128, epochs = 100)


In [0]:
score = model.evaluate(test_set, test_targets, verbose=0)
print("score : ", score)

In [0]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [0]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()